In [13]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util
from torch.utils.data import DataLoader

In [14]:
import pandas as pd

example_hash = set()
train_examples = []

sent_pairs_df = pd.read_csv('../data/processed/7_5_sent_pairs_20_30.csv')

for sent_1, sent_2, final_score in sent_pairs_df[['sent1', 'sent2', 'final_score']].values:
    train_examples.append(
    InputExample(texts=[sent_1, sent_2], label=round(float(final_score), 2))
)

In [15]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, evaluation
from torch.utils.data import DataLoader

model = SentenceTransformer('./sbert_from_mlm_bert_8')

# Создание и загрузка датасета
train_dataset = SentencesDataset(examples=train_examples, model=model)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

eval_examples = train_examples[:100]

eval_dataset = SentencesDataset(examples=eval_examples, model=model)

In [16]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    examples=eval_examples,
    name='sent_pairs_eval'
)


# Настройка процесса обучения
train_loss = losses.CosineSimilarityLoss(model=model)

# Обучение модели
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    evaluation_steps=100,
    epochs=1)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/31758 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
model.save('./sbert_from_mlm_bert_9')